# Entregable 2
## Consigna
El script del entregable 1 deberá adaptar datos leídos de la API y cargarlos en la tabla creada en la pre-entrega anterior en Redshift.

In [1]:
import os
import psycopg2
import requests
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col, max, lag
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
#!pip install psycopg2-binary

In [3]:
# Postgres and Redshift JDBCs
driver_path = "/home/coder/working_dir/driver_jdbc/postgresql-42.2.27.jre7.jar"

os.environ['PYSPARK_SUBMIT_ARGS'] = f'--driver-class-path {driver_path} --jars {driver_path} pyspark-shell'
os.environ['SPARK_CLASSPATH'] = driver_path

# Create SparkSession 
spark = SparkSession.builder \
        .master("local") \
        .appName("Conexion entre Pyspark y Redshift") \
        .config("spark.jars", driver_path) \
        .config("spark.executor.extraClassPath", driver_path) \
        .getOrCreate()

In [4]:
env = os.environ

## Extracción de datos

In [5]:
def extract_data(symbol):
    try:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={env["API_KEY"]}'
        response = requests.get(url)
        json_data = response.json()

        data_list = []
        for date, values_dict in json_data["Monthly Time Series"].items():
            data = (date, values_dict["1. open"], values_dict["2. high"], values_dict["3. low"], values_dict["4. close"], values_dict["5. volume"])
            data_list.append(data)

        # Crear el DataFrame con todos los datos
        df = spark.createDataFrame(data_list, ["date_from", "1. open", "2. high", "3. low", "4. close", "5. volume"])
        df = df.withColumn("symbol", lit(symbol))
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error de solicitud: {e}")
        return None

In [6]:
data_ibm = extract_data('IBM')
data_aapl = extract_data('AAPL')
data_tsla = extract_data('TSLA')
data = data_ibm.union(data_aapl).union(data_tsla)

## Conexión a la base de datos y creación de tabla

In [7]:
# Connect to Redshift using psycopg2
conn = psycopg2.connect(
    host=env['AWS_REDSHIFT_HOST'],
    port=env['AWS_REDSHIFT_PORT'],
    dbname=env['AWS_REDSHIFT_DBNAME'],
    user=env['AWS_REDSHIFT_USER'],
    password=env['AWS_REDSHIFT_PASSWORD']
)

In [8]:
cursor = conn.cursor()
cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {env['AWS_REDSHIFT_SCHEMA']}.{env['AWS_REDSHIFT_TABLE']} (
            "date_from" VARCHAR(10),
            "1. open" VARCHAR(10),
            "2. high" VARCHAR(10),
            "3. low" VARCHAR(10),
            "4. close" VARCHAR(10), 
            "5. volume" VARCHAR(10),
            symbol VARCHAR(10) distkey
        ) sortkey(date_from);
    ''')
conn.commit()
cursor.close()
print("Table created!")

Table created!


## Verificación de duplicados

In [9]:
# Verificación de duplicados
total_rows = data.count()
distinct_rows = data.dropDuplicates().count()

# Comparar la cantidad de filas antes y después de eliminar los duplicados
if total_rows == distinct_rows:
    print("El DataFrame no tiene duplicados.")
else:
    print("El DataFrame tiene duplicados.")

El DataFrame no tiene duplicados.


## Creación de columna variación mensual

In [10]:
window_spec = Window.partitionBy('symbol').orderBy('date_from')
data = data.withColumn('monthly variation (%)', (col('`4. close`') - lag('`4. close`').over(window_spec)) / lag('`4. close`').over(window_spec) * 100)

## Carga de datos

In [11]:
data.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"{env['AWS_REDSHIFT_SCHEMA']}.{env['AWS_REDSHIFT_TABLE']}") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

## Consulta

In [12]:
# Query Redshift using Spark SQL
query = f"select * from {env['AWS_REDSHIFT_SCHEMA']}.{env['AWS_REDSHIFT_TABLE']} limit 10"
data = spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{env['AWS_REDSHIFT_HOST']}:{env['AWS_REDSHIFT_PORT']}/{env['AWS_REDSHIFT_DBNAME']}") \
    .option("dbtable", f"({query}) as tmp_table") \
    .option("user", env['AWS_REDSHIFT_USER']) \
    .option("password", env['AWS_REDSHIFT_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [13]:
data.printSchema()
data.show()

root
 |-- date_from: string (nullable = true)
 |-- 1. open: string (nullable = true)
 |-- 2. high: string (nullable = true)
 |-- 3. low: string (nullable = true)
 |-- 4. close: string (nullable = true)
 |-- 5. volume: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- monthly variation (%): double (nullable = true)

+----------+--------+--------+--------+--------+---------+------+---------------------+
| date_from| 1. open| 2. high|  3. low|4. close|5. volume|symbol|monthly variation (%)|
+----------+--------+--------+--------+--------+---------+------+---------------------+
|1999-12-31|101.0000|118.0000| 91.0600|102.8100| 84091200|  AAPL|                 null|
|2000-01-31|104.8700|121.5000| 86.5000|103.7500|112099800|  AAPL|   0.9143079466977897|
|2000-02-29|104.0000|119.9400| 97.0000|114.6200| 65355200|  AAPL|   10.477108433734944|
|2000-03-31|118.5600|150.3800|114.0000|135.8100| 77663900|  AAPL|   18.487175013086716|
|2000-04-28|135.5000|139.5000|104.8700|124.0600| 